### 실습 개요
- 다양한 속성을 가진 이미지 array(3차원)를 생성하고 시각화합니다.
- 다양한 이미지 array들 간에 유사도를 측정합니다.
- Video array의 연속되는 두 이미지(frame)들 간의 유사도의 의미를 분석합니다.

### 사전준비
- functions.py 안에 mp4 파일 decoding 함수들을 import
- numpy를 import

In [1]:
# functions.py 에 미리 정의된 함수들을 import 한다
import functions as fs

# numpy를 import 한다
import numpy as np

# matplotlib.pyplot을 import 한다
import matplotlib.pyplot as plt

- mp4 file의 video data를 ndarray(N-dimensional Array) 변수에 저장하기

In [2]:
"""
함수 video_2_ndarray 를 사용하여 ../media/SampleVideo_640x360_5mb.mp4 의 video data를
ndarray 에 저장한다.
"""
video_array, _, _ = fs.video_2_ndarray('../media/SampleVideo_640x360_5mb.mp4')

### 따라 해보기 #1. 다차원 array(여기서는 video frame)들 간 유사도 측정하기

#### 1. 다양한 test image array 들을 생성
- 세로 : 360, 가로 : 640 사이즈의 다양한 test image array 들을 생성

In [3]:
# noisy image 2개
noise1 = np.random.randint(0, 255, size=(360, 640, 3), dtype=np.uint8)
noise2 = np.random.randint(0, 255, size=(360, 640, 3), dtype=np.uint8)

# R, G, B 값 중 하나는 최대(255), 나머지는 최소(0)
red = np.full((360, 640, 3), (255, 0, 0), dtype=np.uint8)
blue = np.full((360, 640, 3), (0, 0, 255), dtype=np.uint8)

# R, G, B 값이 254로 매우 밝은색(하얀색)
light = np.full((360, 640, 3), (254, 254, 254), dtype=np.uint8)

# R, G, B 값이 1로 매우 어두운색(검은색)
dark = np.full((360, 640, 3), (1, 1, 1), dtype=np.uint8)

In [ ]:
# Numpy Array의 shape와 size 확인
print(f"noise1 shape : {noise1.shape}, size : {noise1.size}")
print(f"red shape : {red.shape}, size : {red.size}")
print(f"light shape : {light.shape}, size : {light.size}")
print(f"dark shape : {dark.shape}, size : {dark.size}")

#### 2. test image array들 출력

In [ ]:
images = [noise1, noise2, red, blue, light, dark]

fs.plot_images(images, rows=3, columns=2)

#### 3. test image들 간의 유사성을 측정하기 위해 test image들을 가공하기
- 평탄화(Flattening)
    - 다차원 배열을 벡터(1차원)로 변환하여 벡터의 내적, 노름(norm)등 벡터 연산이 가능하게 함
- 최소-최대 정규화(Min-Max Normalization)
    - 가장 작은 값을 0, 가장 큰 값을 1로 두고, 나머지 값들은 비율을 맞춰서 모두 0과 1 사이의 값으로 스케일링. 

In [7]:
# 1. 다차원를 1차원으로 평탄화(Flattening)한다.
# 2. 그 결과를 255로 나누어 각 항목의 값이 0 ~ 1 사이의 float 값을 가지도록 최소-최대 정규화(Min-Max Normalization)한다.
#    공식 : (x - min) / (max - min) = (x - 0) / (255 - 0) = x / 255

noisy1_vector = noise1.reshape(-1)/255
noisy2_vector = noise2.reshape(-1)/255

red_vector = red.reshape(-1)/255
blue_vector = blue.reshape(-1)/255

light_vector = light.reshape(-1)/255
dark_vector = dark.reshape(-1)/255

#### 4. 쉽고 간단한 유사도 측정 방법 cosine similatiry

In [8]:
from numpy import dot
from numpy.linalg import norm

# cosine similatiry 함수
def cos_sim(A, B):
    return dot(A, B) / (norm(A) * norm(B))

- 그림 : cosine similarity 함수 이해

    ![Alt text for broken image link](../resources/cosine-similarity-vectors.jpg)
    - image vector의 모든 값은 0 ~ 1 이므로 마이너스 유사도(opposite vectors)는 나타나지 않음. 최소값은 0

#### 5. cosine similatiry 함수를 사용한 유사도 구하기

In [ ]:
# dark와 light 간 유사도
print('dark와 light 간 유사도 : ', cos_sim(dark_vector, light_vector))

# 두 noise 간 유사도
print('두 noise 간 유사도 : ', cos_sim(noisy1_vector, noisy2_vector))

# red와 blue 간 유사도
print('red와 blue 간 유사도 : ', cos_sim(red_vector, blue_vector))

### 따라 해보기 #2. 동영상의 연속되는 두 frame들간의 유사도 측정하기

#### 1. 연속되는 두 frame들간의 유사도를 구해서 차례로 list에 저장

In [ ]:
# 방법 #1 : 일반적인 반복문 사용

# 이전 프레임 vector  초기화
prev_vector = None

# 유사도 값을 저장할 list 변수 초기화
similarity_list = []

# 각 프레임별로 처리
for frame in video_array:

    # 평탄화(Flattening) 및 최소-최대 정규화(Min-Max Normalization)
    # 1차원으로 reshape하고 255로 나눔
    current_vector = frame.reshape(-1) / 255

    # 첫번째 frame인 경우 건너뜀
    if prev_vector is not None:
        # 유사도 계산 및 similarity_list에 append
        similarity = cos_sim(prev_vector, current_vector)
        similarity_list.append(similarity)
            
    # 이전 프레임 vector를 현재 vector로 업데이트
    prev_vector = current_vector.copy()

print('frame하나의 vector size : ',current_vector.size)
print('similarity_list : ',similarity_list)

In [14]:
# 방법 #2 : zip을 사용한 반복문

# 유사도 값을 저장할 list 변수 초기화
similarity_list = []

# 연속되는 2개의 프레임을 처리
for frame_1, frame_2 in zip(video_array[:-1], video_array[1:]):

    # 평탄화(Flattening) 및 최소-최대 정규화(Min-Max Normalization)
    # 1차원으로 reshape하고 255로 나눔
    vector_1 = frame_1.reshape(-1) / 255
    vector_2 = frame_2.reshape(-1) / 255

    # 유사도 계산 및 similarity_list에 append
    similarity = cos_sim(vector_1, vector_2)
    similarity_list.append(similarity)

In [11]:
# 방법 #3 : 컴프리헨션(Comprehension) 사용

# 평탄화(Flattening) 및 최소-최대 정규화(Min-Max Normalization)
frame_cnt = video_array.shape[0]
modi_video = video_array.reshape(frame_cnt, -1) / 255

# 유사도 list 생성 (i번째 frame과 i+1번째 frame의 유사도)
similarity_list = [ cos_sim(modi_video[i], c) for i, c in enumerate(modi_video[1:]) ]

In [ ]:
# similarity_list 꺾은선 그래프 그리기

plt.figure(figsize=(10, 6))
plt.plot(range(len(similarity_list)), similarity_list, marker='o', linestyle='-', color='b')
plt.xlabel('Frame Number')
plt.ylabel('Similarity')
plt.title('Similarity vs Frame Number')
plt.grid(True)

#### 2. 연속되는 frame들 중 유사도가 낮은 frame들 출력하기

In [ ]:
# 낮은 유사도 임계치
lower_sim_threshold = 0.9

for index, value in enumerate(similarity_list):

    # 임계치 이하의 두 frame들을 출력
    if value <= lower_sim_threshold:

        frames = [video_array[index], video_array[index+1]]
        titles = ['frame#'+str(index), 'frame#'+str(index+1)]

        fs.plot_images(frames, rows=1, columns=2, titles=titles)